In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import tensorflow as tf

Using TensorFlow backend.


In [2]:
img_width, img_height = 224, 224

train_data_dir = 'train/'
validation_data_dir = 'test/'



nb_train_samples = 75
nb_validation_samples = 50
#epochs = 50
#batch_size = 16
epochs = 10
batch_size = 10

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [4]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])



train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)


test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

history=model.fit_generator(train_generator,steps_per_epoch=100,epochs=epochs,
                            validation_data=validation_generator,validation_steps=100)

Found 2637 images belonging to 2 classes.
Found 660 images belonging to 2 classes.
Epoch 1/10
100/100 [==============================] - 140s 1s/step - loss: 0.7648 - accuracy: 0.5430 - val_loss: 0.6144 - val_accuracy: 0.5640
Epoch 2/10
100/100 [==============================] - 149s 1s/step - loss: 0.6439 - accuracy: 0.6630 - val_loss: 0.3129 - val_accuracy: 0.5890
Epoch 3/10
100/100 [==============================] - 131s 1s/step - loss: 0.6018 - accuracy: 0.7213 - val_loss: 0.5166 - val_accuracy: 0.7850
Epoch 4/10
100/100 [==============================] - 120s 1s/step - loss: 0.5685 - accuracy: 0.7390 - val_loss: 0.5107 - val_accuracy: 0.6870
Epoch 5/10
100/100 [==============================] - 126s 1s/step - loss: 0.5437 - accuracy: 0.7290 - val_loss: 0.4884 - val_accuracy: 0.7750
Epoch 6/10
100/100 [==============================] - 132s 1s/step - loss: 0.5389 - accuracy: 0.7570 - val_loss: 0.6349 - val_accuracy: 0.7450
Epoch 7/10
100/100 [==============================] - 137s 

In [5]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
activation_6 (Activation)    (None, 222, 222, 32)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 109, 109, 32)      9248      
_________________________________________________________________
activation_7 (Activation)    (None, 109, 109, 32)      0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 54, 54, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 52, 52, 64)       

In [6]:
keras_file = "model_coba.h5"
model.save(keras_file)

In [7]:
converter = tf.lite.TFLiteConverter.from_keras_model_file(keras_file)
tflite_model = converter.convert()

Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 10 variables.
INFO:tensorflow:Converted 10 variables to const ops.


In [8]:
open("converted_model.tflite", "wb").write(tflite_model)

11193356